<a href="https://colab.research.google.com/github/marie0501/fine_tuning_transformers/blob/main/BERT_for_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone --branch dev https://github.com/rolysr/medical-knowledge-discoverer


Cloning into 'medical-knowledge-discoverer'...
remote: Enumerating objects: 445, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 445 (delta 51), reused 73 (delta 32), pack-reused 339
Receiving objects: 100% (445/445), 1.37 MiB | 4.97 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [5]:
%cd medical-knowledge-discoverer/
!ls

/content/medical-knowledge-discoverer
datasets  main.py  ontology   requirements.txt
LICENSE   models   README.md  utils


In [6]:
!pip install -r requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 25.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of es-core-news-sm to determine which version is compatible with other requirements. This could 

In [7]:
%%capture
pip install transformers

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

from pathlib import Path
from utils.anntools import Collection
import csv

import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

In [9]:
c = Collection()
for fname in Path("datasets/train").rglob("*.txt"):
    c.load(fname)

len(c.sentences)
c.sentences[0].keyphrases


[Keyphrase(text='outbreak', label='Concept', id=1, attr=[]),
 Keyphrase(text='coronavirus disease-2019', label='Concept', id=2, attr=[]),
 Keyphrase(text='COVID-19', label='Concept', id=3, attr=[]),
 Keyphrase(text='novel', label='Concept', id=4, attr=[]),
 Keyphrase(text='severe acute respiratory syndrome coronavirus 2', label='Concept', id=5, attr=[]),
 Keyphrase(text='SARS-CoV-2', label='Concept', id=6, attr=[]),
 Keyphrase(text='serious', label='Concept', id=7, attr=[]),
 Keyphrase(text='threat', label='Concept', id=8, attr=[]),
 Keyphrase(text='public health', label='Concept', id=9, attr=[])]

## Leer CSV Data

## Inicializar Tokenizer


In [10]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

## Crear clase Dataset

In [13]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(label_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(label_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, collection:Collection):
      lb = []
      txt = []
      for sentence in collection.sentences:
          new_sentence = sentence.text.replace('(','').replace(')','')
          txt.append(new_sentence)
          labels = [-100 for i in range(len(new_sentence.split(' ')))]
          keyphrases = sentence.keyphrases
          sentence_list = new_sentence.split(' ')
          count = 0
          k = 0
          for keyphrase in sentence.keyphrases:
              index = new_sentence.find(keyphrase.text)   
              while count + k < index:
                  count += len(sentence_list[k])
                  k+=1
              keyphrases_list = keyphrase.text.split(' ')
              if k < len(sentence_list) and sentence_list[k] == keyphrases_list[0]:
                  labels[k]= 'B-' + keyphrase.label 
                  count += len(keyphrases_list[0])
                  i = 1
                  while i < len(keyphrases_list):
                      labels[k + i] = 'I-' + keyphrase.label 
                      count += len(keyphrases_list[i])
                      i += 1
                  k +=i
          lb.append(labels.copy())
        
    
      self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]

      print(self.texts[0])
      self.labels = [align_label(i,j) for i,j in zip(txt, lb)]
      print(self.labels[0])

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

d = DataSequence(c)


{'input_ids': tensor([[  101, 10117, 14978, 63831, 10108, 31206, 37715, 10251, 21911,   118,
         19493, 32992, 90939, 11490,   118, 10270, 19513, 10155,   169, 14671,
         34844, 75730, 52812, 56610, 21493, 38117, 31206, 37715, 10251,   123,
         97404, 10731,   118, 13098, 11779,   118,   123, 10124, 39327, 10230,
           169, 32342, 41256, 10114, 11683, 17004,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  